# merge segmentations UKB, KORA, NAKO


In [1]:
import os

import nibabel  # for loading and saving nifty files
import torch
from math import ceil, floor
from PIL import Image
from torch.utils.data import Dataset
from torchvision.transforms import Pad
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import nrrd
from nibabel.affines import from_matvec, to_matvec, apply_affine

In [2]:
print(os.getcwd())

/home/anne/phd/projects/whole_body/whole_body_segmentation/quickNAT_pytorch/create_datasets


## combination of segmentation files:
creates new segmentation image of the same size as the merged volume  
changes the labels of the segmentation maps according to label list below  
also reads header file of .nrrd files.  
merges segmentations according to header['space origin']


In [16]:
def combine_seg(seg_files, data_dir):
    #data_dir = seg_dir
    img_file = [f for f in seg_files if 'input_scan.nii.gz' in f and not 'combined' in f]
    print('img file: ', img_file)
    if len(img_file) == 1:
        img_file = img_file[0]
    else:
        print('less or more than 1 img file found')

    print('data_dir ', data_dir)
    print('img_file ', img_file)
    
    print('load and reorient image')
    img, data, img_h = reorient_image(os.path.join(data_dir, img_file), is_label=False)
    
    #ONLY FOR KORA!!!!
    #data = np.flip(np.moveaxis(data, 2, 1))  # 012 -> 021
    
    #img = nibabel.load(os.path.join(data_dir, img_file))
    #print(img)
    #img_h = img.header
    #print(img_h)
    print('x: ', img_h['qoffset_x'])
    print('y: ', img_h['qoffset_y'])
    print('z: ', img_h['qoffset_z'])

    # somehow this is wrong. x, and y should be positive and z negative

    segmentations = {'liver': [],
                    'spleen': [],
                    'kidney_r': [],
                    'kidney_l': [],
                    'adrenal_l': [],
                    'adrenal_r': [],
                    'pancreas': [],
                    'gallbladder': []}
    for file in seg_files:
        if "nrrd" in file:
            print('load and reorient segmentation')
            seg, seg_data, header = reorient_image(os.path.join(data_dir, file), is_label=True)
            
            #seg, header = nrrd.read(os.path.join(data_dir, file))

            # change labels according to label list above. liver stays the same, as it gets label=1         
            seg, seg_label = change_label(seg_data, file)

            # print out some important header information:
            ##print(file)
            
            print('segm shape: ', seg_data.shape)
            print('x: ', header['qoffset_x'])
            print('y: ', header['qoffset_y'])
            print('z: ', header['qoffset_z'])
            
            #space_directions = header['space directions']
            #space_origin = header['space origin']
            #sizes = header['sizes']
            #print('z: ', space_origin[2])
            #print('size z: ', sizes[2])
            #print('spacing z: ', space_directions[2,2])

            segmentations[seg_label] = [seg_data,header] # append tuple of seg and header, need header later
    #print(segmentations)

    # check for overlaps:
    try:
        s1, s2 = check_for_overlaps([segmentations['liver'][0], 'liver'], [segmentations['gallbladder'][0], 'gallbladder'])
        segmentations['liver'][0] = s1
        segmentations['gallbladder'][0] = s2
    except IndexError as err :
        print('doesnt have one of the segmentations liver or gallbladder')
    except ValueError as err:
        print('liver and gallbladder sizes differ')
    
    
    #print(segmentations)
    try:
        s1, s2 = check_for_overlaps([segmentations['kidney_l'][0], 'kidney_l'], [segmentations['spleen'][0], 'spleen'])
   
        segmentations['kidney_l'][0] = s1
        segmentations['spleen'][0] = s2
    except IndexError as err: 
        print('doesnt have one of the segmentations kidney l or spleen')
    except ValueError as err:
        print('spleen and kidney sizes differ')

    
    #s1, s2 = check_for_overlaps([segmentations['kidney_l'][0], 'kidney_l'], [segmentations['pancreas'][0], 'pancreas'])
    #segmentations['kidney_l'][0] = s1
    #segmentations['pancreas'][0] = s2
    
    ##s1, s2 = check_for_overlaps([segmentations['spleen'][0], 'spleen'], [segmentations['pancreas'][0], 'pancreas'])
    #segmentations['spleen'][0] = s1
    #segmentations['pancreas'][0] = s2
    
    try:
        s1, s2 = check_for_overlaps([segmentations['adrenal_r'][0], 'adrenal_r'], [segmentations['gallbladder'][0], 'gallbladder'])
        segmentations['adrenal_r'][0] = s1
        segmentations['gallbladder'][0] = s2
    except IndexError as err:
        print('doesnt have one of the segmentations adrenal r or gallbladder')
    except ValueError as err:
        print('adrenal and gallbladder sizes differ')
    
    try:
        s1, s2 = check_for_overlaps([segmentations['adrenal_l'][0], 'adrenal_l'], [segmentations['pancreas'][0], 'pancreas'])
        segmentations['adrenal_l'][0] = s1
        segmentations['pancreas'][0] = s2
    except IndexError as err:
        print('doesnt have one of the segmentations adrenal l or pancreas')
    except ValueError as err:
        print('adrenal and pancreas sizes differ')
        
    #print(len(segmentations))
    num_segm = len(segmentations)
    if num_segm < 8:
        print('less than 8 segmentations')

    #segm_sorted = sorted(segmentations, key=lambda seg: seg[1]['space origin'][2], reverse=True)

    comb_segm = np.zeros_like(img.get_data())
    print('final shape: ', comb_segm.shape)

    img_dim_v = img.shape

    
    for label in segmentations:
        print(label)
        
        #print('reorient seg')
        #seg, segm, header = reorient_image(seg, is_label=True)
        try: 
            segm = segmentations[label][0]
            header = segmentations[label][1]
        except IndexError as ierr:
            print('doesnt have the segmentation for ', label)
            continue
        #print(segm)
        #print(header)
        
        #segm_off = header['space origin']
        #segm_dim_v = header['sizes']
        #segm_spacing = abs(np.diag(header['space directions']))
        
        segm_spacing = abs(header['pixdim'][1:4])
        segm_dim_v = segm.shape
        # CAVE: not sure if abs is correct here
        segm_offx = abs(header['qoffset_x'])
        segm_offy = abs(header['qoffset_y'])
        segm_offz = header['qoffset_z']


        segm_offset = np.array([segm_offx, segm_offy, segm_offz])
        # if same size, just add the segmentation
        print('im dim v: ', img_dim_v)
        print('segm dim v: ', segm_dim_v)
            
            
        if (segm_dim_v == img_dim_v):
            print('same')
            comb_segm += segm
        else:
            print('im dim v: ', img_dim_v)
            print('segm dim v: ', segm_dim_v)

            im_spacing = abs(img_h['pixdim'][1:4])
            print('im spacing: ', im_spacing)
            print('segm spacing: ', segm_spacing)

            im_dim_w = img_dim_v * im_spacing
            segm_dim_w = segm_dim_v * segm_spacing

            print('im dim w: ', im_dim_w)
            print('segm dim w: ', segm_dim_w)

            # correction of the wrong information in header file
            im_offx = abs(img_h['qoffset_x'])
            im_offy = abs(img_h['qoffset_y'])
            im_offz = img_h['qoffset_z']


            im_offset = np.array([im_offx, im_offy, im_offz])
            im_start = im_offset
            im_end = np.array([im_start[0]-im_dim_w[0], im_start[1]-im_dim_w[1], im_start[2]+im_dim_w[2]])

            segm_start = segm_offset
            segm_end = np.array([segm_start[0]-segm_dim_w[0], segm_start[1]-segm_dim_w[1], segm_start[2]+segm_dim_w[2]])

            print('im off: ', im_offset)
            print('segm off: ', segm_offset)

            print('im start: ', im_start)
            print('im end: ', im_end)
            print('segm start: ', segm_start)
            print('segm end: ', segm_end)

            start_diff_w = abs(im_start - segm_start)
            end_diff_w = abs(im_end - segm_end)
            print('start diff w: ', start_diff_w)

            print('end_diff w ', end_diff_w)

            start_diff_v = np.round(start_diff_w / segm_spacing).astype(int)
            print('start diff v: ', start_diff_v)

            end_diff_v = np.round(end_diff_w / segm_spacing).astype(int)
            print('end diff v: ', end_diff_v)

            segm_end_v = img_dim_v - end_diff_v
            print('segm end v: ', segm_end_v)

            segm_end_x = segm_end_v[0]
            segm_end_y = segm_end_v[1]
            segm_end_z = segm_end_v[2]
            segm_start_x = segm_end_x - segm_dim_v[0]
            segm_start_y = segm_end_y - segm_dim_v[1]
            segm_start_z = segm_end_z - segm_dim_v[2]
            print('segm start v: ', segm_start_x, segm_start_y, segm_start_z)

            comb_segm[segm_start_x:segm_end_x, segm_start_y:segm_end_y, segm_start_z:segm_end_z] += segm

    empty_header = nibabel.Nifti1Header()
    new_img_nii = nibabel.Nifti1Image(comb_segm, img.affine, img_h)
    #new_img_nii.header['pixdim'] = img_h['pixdim']

    #print(new_img_nii.header)
    new_file_path = os.path.join(data_dir,'combined_segmentation.nii.gz')
    nibabel.save(new_img_nii, new_file_path)
    
    # also save reoriented image:
    new_file_path = os.path.join(data_dir,'input_scan.nii.gz')
    
    nibabel.save(img, new_file_path)



### changes labels according to this list:
liver = 1  
spleen = 2  
kidney_r = 3  
kidney_l = 4  
adrenal_r = 5  
adrenal_l = 6  
pancreas = 7  
gallbladder = 8

Caution: does not consider spelling mistakes ;) have to manually check first if all files are named correctly


In [4]:
def change_label(seg, f):
    print(f)
    seg_label = None
    if 'liver' in f or 'Liver' in f:
        print('found liver')
        seg[seg == 1] = 1
        seg_label = 'liver'
        
    elif 'spleen' in f or 'Spleen' in f:
        print('found spleen')
        seg[seg == 1] = 2
        seg_label = 'spleen'
    elif ('kidney' in f or 'Kidney' in f) and ('right' in f  or 'Right' in f):
        seg[seg==1] = 3
        print('found right kidney')
        seg_label = 'kidney_r'
        
    elif ('kidney' in f or 'Kidney' in f) and ('left' in f  or 'Left' in f): 
        seg[seg==1] = 4
        print('found left kidney')
        seg_label = 'kidney_l'

    
    elif ('adrenal' in f or 'Adrenal' in f) and ('right' in f  or 'Right' in f): 
        seg[seg==1] = 5
        print('found right adrenal')
        seg_label = 'adrenal_r'
        
    elif ('adrenal' in f or 'Adrenal' in f) and ('left' in f  or 'Left' in f):
        seg[seg==1] = 6
        print('found left adrenal')
        seg_label = 'adrenal_l'
        
    elif ('pancreas' in f or 'Pancreas' in f):
        seg[seg==1] = 7
        print('found pancreas')
        seg_label = 'pancreas'
    
    elif ('gallbladder' in f or 'Gallbladder' in f):
        seg[seg==1] = 8
        print('found gallbladder')
        seg_label = 'gallbladder'
    else:
        seg[seg==1] = 0
        print('found other organ, setting labels to 0')
        
    return seg, seg_label

## checks for overlaps of segmentations

even though Oksana checked manually if some segmentations overlap, there might be some few voxels that still overlap.

This function just checks the organs that have highest risk of overlapping ;)  

main overlaps to be expected between gallbladder and liver  

CAVE: this only works if the segmentations have the same shape... (so far I didn't encounter any overlaps when the segmentation shapes mismatched.. but of course this should be handled better)

In [5]:
def check_for_overlaps(seg1, seg2):
    seg1, seg1_label = seg1
    seg2, seg2_label = seg2
    print('seg 1 shape', seg1.shape)
    print('seg 2 shape', seg2.shape)
    print(seg1_label, seg2_label)
    
    if seg1_label == 'liver' and seg2_label == 'gallbladder':
        overlap = seg1 * seg2
        overlap = np.where(overlap == 8)
        print(overlap)
        seg1[overlap] = 0
        
    if seg1_label == 'kidney_l' and seg2_label == 'spleen':
        overlap = seg1 * seg2
        overlap = np.where(overlap == 8)
        print(overlap)
        seg1[overlap] = 0
        
    if seg1_label == 'adrenal_r' and seg2_label == 'gallbladder':
        overlap = seg1 * seg2
        overlap = np.where(overlap == 24)
        print(overlap)
        seg1[overlap] = 0
    
    if seg1_label == 'adrenal_l' and seg2_label == 'pancreas':
        overlap = seg1 * seg2
        overlap = np.where(overlap == 42)
        print(overlap)
        seg1[overlap] = 0
    return seg1, seg2
        

In [6]:
def reorient_image(in_file, axes='RAS', translation_params=(0, 0, 0), is_label=False):
    # Global parameters
    POSSIBLE_AXES_ORIENTATIONS = [
        "LAI", "LIA", "ALI", "AIL", "ILA", "IAL",
        "LAS", "LSA", "ALS", "ASL", "SLA", "SAL",
        "LPI", "LIP", "PLI", "PIL", "ILP", "IPL",
        "LPS", "LSP", "PLS", "PSL", "SLP", "SPL",
        "RAI", "RIA", "ARI", "AIR", "IRA", "IAR",
        "RAS", "RSA", "ARS", "ASR", "SRA", "SAR",
        "RPI", "RIP", "PRI", "PIR", "IRP", "IPR",
        "RPS", "RSP", "PRS", "PSR", "SRP", "SPR"
    ]

    #Load the image to rectify
    if in_file.split('.')[-1] == 'nrrd':
        print('NRRD file received, converting to nifti!!!')
        image = nrrd_to_nifti(in_file)
    else:
        image = nibabel.load(in_file)
        print(image.get_affine())
    #if not is_label:
        #volid = in_file.split('/')[-2]
        #PreProcess.save_nibabel(volume=image.get_fdata(), header=image.header, filename=f'{volid}_vol_after_loaded')

    # Get the axes automatically.

    header = image.header
    #print(header)    
    axes = nibabel.orientations.aff2axcodes(header.get_best_affine())
    axes = "".join(list(axes))

    # Check that a valid input axes is specified
    if axes not in POSSIBLE_AXES_ORIENTATIONS:
        raise ValueError("Wrong coordinate system: {0}.".format(axes))

    rotation = swap_affine(axes)
    det = np.linalg.det(rotation)
    if det != 1:
        raise Exception("Rotation matrix determinant must be one "
                        "not '{0}'.".format(det))

    affine = image.get_affine()
    # Get the trnaslation to apply
    translation = np.eye(4)
    # Get the input image affine transform
    # s, t = to_matvec(affine)
    # print(affine, s, t, tuple(t))
    # print(header.get_best_affine())
    # volume_translations = [-216, -178.9, -664.5]
    # current_t = [-119, -106, -503]
    # intended_t = tuple(np.subtract(volume_translations, t))
    # print(intended_t)
    translation[:3, 3] = translation_params  # tuple(t) #if not is_label else translation_params[:3, 3]
    # print(affine, data.shape)

    # source_orientation = nb.orientations.io_orientation(affine)
    # transformation_mat = nb.orientations.ornt_transform(source_orientation, PreProcess.get_target_orientation())
    # data = nb.orientations.apply_orientation(data, transformation_mat)
    # print(data.shape)

    # Apply the rotation to set the image in the RAS coordiante system
    transformation = np.dot(np.dot(rotation, affine), translation)
    # print('transformation mat:', transformation)
    # image.set_qform(transformation)
    image.set_sform(transformation)


    data = image.get_data()
    # print('after transformation:', data.shape)
    # affine = image.get_affine()
    # s, t = to_matvec(affine)
    # print(affine, s, t, tuple(t))
    # translation = np.eye(4)
    # translation[:3, 3] = tuple(-t)
    # image.set_sform(translation)
    # data = image.get_data()
    # print('after translation:', data.shape)

    # array = image.get_data()
    # print(image.get_affine(), array.shape)

    header = image.header
    new_axes = nibabel.orientations.aff2axcodes(header.get_best_affine())
    new_axes = "".join(list(new_axes))
    # 216.09375, 178.90625, -664.5
    print(f"{axes} transformed to {new_axes}")
    # print(data.shape, header.get_best_affine())
    # print('-' * 39)
    # if not is_label:
    #     return image, data, header, translation
    return image, data, header

In [7]:
def swap_affine(axes):
    CORRECTION_MATRIX_COLUMNS = {
        "R": (1, 0, 0),
        "L": (-1, 0, 0),
        "A": (0, 1, 0),
        "P": (0, -1, 0),
        "S": (0, 0, 1),
        "I": (0, 0, -1)
    }

    if axes not in ['RSP', 'LIP', 'RAS', 'LPS']:
        raise Exception(
            f'Unknown axes passed for affine transformation! Please add transformation for {axes} manually.')
    rotation = np.eye(4)
    rotation[:3, 0] = CORRECTION_MATRIX_COLUMNS[axes[0]]
    rotation[:3, 1] = CORRECTION_MATRIX_COLUMNS[axes[1]]
    rotation[:3, 2] = CORRECTION_MATRIX_COLUMNS[axes[2]]
    # print(rotation)
    if axes == "RSP":
        rotation = np.array([[1., 0., 0., 0.],
                             [0., 0., 1., 0.],
                             [0., -1., 0., 0.],
                             [0., 0., 0., 1.]])
        # print(rotation)
    elif axes == "LPS":
        rotation = np.array([
            [-1., 0., 0., 0.],
            [0., -1., 0., 0.],
            [0., 0., 1., 0.],
            [0., 0., 0., 1.]
        ])
        # print(rotation)
    return rotation


In [8]:
def nrrd_to_nifti(file_path):
    #output_file_path = os.path.join(file_path,'test')+'out.nii.gz'

    data, header = nrrd.read(file_path)

    if len(data.shape) == 4:
        data = data[:, :, :, 0]

    affine = header['space directions']
    affine = affine[:3, :3]
    origins = header['space origin']
    origins = origins[:3]
    t_mat = from_matvec(affine, origins)
    img = nibabel.Nifti1Image(data, t_mat)
    #nibabel.save(img, output_file_path)
    #img = nibabel.load(output_file_path)

    return img

## Calls the combine_seg function. 

change data_dir accordingly. Expects all segmentations and the input scan (in case of UKB the combined input file ...comb_01.nii.gz) in .nrrd or .nii.gz format

seg_dir = directory of the segmentation files, also should include the merged volume ...comb_01.nii.gz

In [18]:
data_dir = '/home/anne/phd/projects/whole_body/whole_body_segmentation/quickNAT_pytorch/create_datasets/KORA/data'
listFolders =  os.listdir(data_dir)
#print('list folders: ', listFolders)

for folder in listFolders:
    print('processing folder: ', folder)
    seg_dir = os.path.join(data_dir,folder)

    listFiles = os.listdir(seg_dir)
    sequence = 'opp'
    seg_files = [f for f in listFiles if ".nrrd" in f or '.nii.gz' in f]
    #print(listFiles)


    combine_seg(seg_files, seg_dir)
    




processing folder:  2460723
img file:  ['input_scan.nii.gz']
data_dir  /home/anne/phd/projects/whole_body/whole_body_segmentation/quickNAT_pytorch/create_datasets/KORA/data/2460723
img_file  input_scan.nii.gz
load and reorient image
[[   1.70138884    0.            0.         -235.24905396]
 [   0.            1.70138884    0.          -31.96945   ]
 [   0.            0.            1.69999695 -109.47180176]
 [   0.            0.            0.            1.        ]]


/home/anne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
/home/anne/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: DeprecationWarning: get_affine method is deprecated.
Please use the ``img.affine`` property instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0


RAS transformed to RAS
x:  235.24905
y:  109.4718
z:  31.96945
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460723_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Kidney (left).nrrd
found left kidney
segm shape:  (288, 288, 160)
x:  -235.24905
y:  -109.4718
z:  31.96945
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460723_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Kidney(right).nrrd
found right kidney
segm shape:  (288, 288, 160)
x:  -235.24905
y:  -109.4718
z:  31.96945
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460723_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-26.nrrd_Pancreas.nrrd
found pancreas
segm shape:  (288, 288, 160)
x:  -235.24905
y:  -109.4718
z:  31.96945
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460723_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Spleen.nrrd
found spleen

RSP transformed to RAS
2460734_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-4.nrrd_Pancreas.nrrd
found pancreas
segm shape:  (288, 288, 176)
x:  -242.58334
y:  -140.06598
z:  -9.543867
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460734_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-7_Adrenal gland(left).nrrd
found left adrenal
segm shape:  (288, 288, 176)
x:  -242.58334
y:  -140.06598
z:  -9.543867
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2460734_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Liver.nrrd
found liver
segm shape:  (288, 288, 176)
x:  -242.58334
y:  -140.06598
z:  -9.543867
seg 1 shape (288, 288, 176)
seg 2 shape (288, 288, 176)
liver gallbladder
(array([ 83,  83,  83,  84,  85,  85,  86,  86,  86,  87,  87,  88,  88,
        88,  89,  90,  92,  96,  96,  96,  96, 107, 107]), array([188, 188, 189, 188, 185, 186, 183, 185, 185, 182, 184, 182, 182,
       183, 182, 180, 179, 182,

segm shape:  (288, 288, 176)
x:  -245.0179
y:  -144.90784
z:  34.414215
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2458366_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8_Kidney (Left).nrrd
found left kidney
segm shape:  (288, 288, 176)
x:  -245.0179
y:  -144.90784
z:  34.414215
load and reorient segmentation
NRRD file received, converting to nifti!!!
RSP transformed to RAS
2458366_t1_vibe_dixon_cor_caipi6_bh_288_iso_opp_8-18.nrrd_Pancreas.nrrd
found pancreas
segm shape:  (288, 288, 176)
x:  -245.0179
y:  -144.90784
z:  34.414215
seg 1 shape (288, 288, 176)
seg 2 shape (288, 288, 176)
liver gallbladder
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
seg 1 shape (288, 288, 176)
seg 2 shape (288, 288, 176)
kidney_l spleen
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
seg 1 shape (288, 288, 176)
seg 2 shape (288, 288, 176)
adrenal_r gallbladder
(array([], dtype=int64), array([], dtype=int64

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
seg 1 shape (288, 288, 160)
seg 2 shape (288, 288, 160)
kidney_l spleen
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
seg 1 shape (288, 288, 160)
seg 2 shape (288, 288, 160)
adrenal_r gallbladder
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
seg 1 shape (288, 288, 160)
seg 2 shape (288, 288, 160)
adrenal_l pancreas
(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))
final shape:  (288, 288, 160)
liver
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
spleen
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
kidney_r
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
kidney_l
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
adrenal_l
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
adrenal_r
im dim v:  (288, 288, 160)
segm dim v:  (288, 288, 160)
same
pancreas
im dim v:  (288, 288, 160)
